https://www.kaggle.com/c/lish-moa/overview

The Connectivity Map, a project within the Broad Institute of MIT and Harvard, together with the Laboratory for Innovation Science at Harvard (LISH), presents this challenge with the goal of advancing drug development through improvements to MoA prediction algorithms.

What is the Mechanism of Action (MoA) of a drug? And why is it important?

In the past, scientists derived drugs from natural products or were inspired by traditional remedies. Very common drugs, such as paracetamol, known in the US as acetaminophen, were put into clinical use decades before the biological mechanisms driving their pharmacological activities were understood. Today, with the advent of more powerful technologies, drug discovery has changed from the serendipitous approaches of the past to a more targeted model based on an understanding of the underlying biological mechanism of a disease. In this new framework, scientists seek to identify a protein target associated with a disease and develop a molecule that can modulate that protein target. As a shorthand to describe the biological activity of a given molecule, scientists assign a label referred to as mechanism-of-action or MoA for short.

How do we determine the MoAs of a new drug?

One approach is to treat a sample of human cells with the drug and then analyze the cellular responses with algorithms that search for similarity to known patterns in large genomic databases, such as libraries of gene expression or cell viability patterns of drugs with known MoAs.

In this competition, you will have access to a unique dataset that combines gene expression and cell viability data. The data is based on a new technology that measures simultaneously (within the same samples) human cells’ responses to drugs in a pool of 100 different cell types (thus solving the problem of identifying ex-ante, which cell types are better suited for a given drug). In addition, you will have access to MoA annotations for more than 5,000 drugs in this dataset.

As is customary, the dataset has been split into testing and training subsets. Hence, your task is to use the training dataset to develop an algorithm that automatically labels each case in the test set as one or more MoA classes. Note that since drugs can have multiple MoA annotations, the task is formally a multi-label classification problem.

How to evaluate the accuracy of a solution?

Based on the MoA annotations, the accuracy of solutions will be evaluated on the average value of the logarithmic loss function applied to each drug-MoA annotation pair.

If successful, you’ll help to develop an algorithm to predict a compound’s MoA given its cellular signature, thus helping scientists advance the drug discovery process.

In this notebook we will try a Feedforward neural network model.\
We will try to use the new variables defined in the notebook 'MoA_model_lightgbm_PCA'
All data analysis is donne in the "MoA data analysyis" notebook.\

In [1]:
import gc
import time
import pandas as pd
import numpy as np
import re


# Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done!
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F

#Load the data. The path is the same for everyone.
train_features_set = pd.read_csv('../input/lish-moa/train_features.csv')
train_labels_set = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

In [7]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed = 1)

In [8]:
'''
Lets separate train_features and train_targets_scored into train_set and validation_set.
This is important to avoid overfitting.
'''

#Remove control observations.
train_features_without_control = train_features_set[train_features_set['cp_type'] != 'ctl_vehicle'].copy()
train_labels_without_control = train_labels_set.iloc[train_features_without_control.index].copy()

#reset index. 
train_features_without_control.reset_index(inplace = True)
train_features_without_control.drop('index', axis = 1, inplace = True)

train_labels_without_control.reset_index(inplace = True)
train_labels_without_control.drop('index', axis = 1, inplace = True)

train_features_without_control = train_features_without_control.drop(['sig_id' ,'cp_type'], axis = 1)
train_x = train_features_without_control.copy()

train_y = train_labels_without_control.drop(['sig_id'], axis = 1).copy()

# For the test, we will remove just the column 'cp_type'.
# However, we will not delete the observations. 
# Instead, we will create a separate data frame with 'cp_type' so we can set all MoA equal to zero for these observations.

test_features_without_control = test_features.drop(['cp_type'], axis = 1).copy()
test_x = test_features_without_control.drop(['sig_id'], axis = 1).copy()
test_x_control = test_features[['sig_id', 'cp_type']].copy()


#Turn the dosage variable in int
train_x['cp_dose'] = train_x['cp_dose'].str.extract(r"([1-2])", expand = True).astype(np.int8)
test_x['cp_dose'] = test_x['cp_dose'].str.extract(r"([1-2])", expand = True).astype(np.int8)

C:\Users\maxwi\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
def log_loss_mean(y_real, predictions):
    y_real_db = pd.DataFrame(data = y_real)
    
    log_loss_aux = []
    for col in y_real_db:
        log_loss_aux.append(log_loss(y_real_db[col], predictions[col], labels = [0, 1]))
        
    return np.mean(log_loss_aux)


def batchify_data(x_data, y_data, batch_size):
    """Takes a set of data points and labels and groups them into batches."""
    # Only take batch_size chunks (i.e. drop the remainder)
    N = int(len(x_data) / batch_size) * batch_size
    batches = []
    for i in range(0, N, batch_size):
        batches.append({
            'x': torch.tensor(x_data[i:i+batch_size], dtype=torch.float32),
            'y': torch.tensor(y_data[i:i+batch_size], dtype=torch.long
        )})
    return batches


def compute_accuracy(predictions, y):
    """Computes the accuracy of predictions against the gold labels, y."""
    predictions_np = predictions.detach().numpy()
    y_np = y.detach().numpy()
    return np.mean(np.equal(predictions_np, y_np))


# Training Procedure
def train_model(train_data, model, lr=0.01, momentum=0.9, nesterov = False, n_epochs=30, save_model = 'FNN'):
# def train_model(train_data, dev_data, model, lr=0.01, n_epochs=30):
    """Train a model for N epochs given data and hyper-params."""
    # We optimize with SGD
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=nesterov)
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)  #It takes too long! with epochs = 320, lr = 0.1 and batch_size = 32 it did not help.

    for epoch in range(1, n_epochs):
        print("-------------\nEpoch {}:\n".format(epoch))


        # Run **training***
        train_loss, train_acc, train_out = run_epoch(train_data, model.train(), optimizer)
        print('Train loss: {:.6f} | Train accuracy: {:.6f}'.format(train_loss, train_acc))

        # Save model
        torch.save(model, save_model + '.pt')
    return 


def run_epoch(data, model, optimizer):
    """Train model for one pass of train data, and return loss, acccuracy"""
    # Gather losses
    losses = []
    batch_accuracies = []

    # If model is in train mode, use optimizer.
    is_training = model.training

    # Iterate through batches
    for batch in tqdm(data):
        # Grab x and y
        x, y = batch['x'], batch['y']
       
        # Get output predictions
        out = model(x)
        y = y.type_as(out)
        
        
        # Predict and store accuracy      
        predictions = torch.round(out)
        batch_accuracies.append(compute_accuracy(predictions, y))


        # Compute loss
        loss = F.binary_cross_entropy(out, y)
        losses.append(loss.data.item())

        # If training, do an update.
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Calculate epoch level scores
    avg_loss = np.mean(losses)
    avg_accuracy = np.mean(batch_accuracies)
    return avg_loss, avg_accuracy, out


Lets run our final models with one, two, and three hidden layers.

In [13]:
# Final model 1 hidden layer

X_train = train_x.to_numpy(copy = True)
Y_train = train_y.to_numpy(copy = True)

X_holdout = test_x.to_numpy(copy = True)   
X_holdout = torch.tensor(X_holdout, dtype=torch.float32)


n_epochs = 290               #Hyperparameter
batch_size = 32             #Hyperparameter
train_batches = batchify_data(X_train, Y_train, batch_size)
X_train_full = torch.tensor(X_train, dtype=torch.float32)


#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.5*X_train.shape[1])), 
                    nn.ReLU6(),  
                    nn.Linear(int(0.5*X_train.shape[1]), 206),  
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.1 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_1')

# Get output.
# Evaluate the model on test data.
out_train = model(X_train_full) 
out_holdout = model(X_holdout)  

out_train = out_train.detach().numpy()
out_train_layers_1 = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_holdout = out_holdout.detach().numpy()
# Make a matrix with zeros when control.
treatment_flag = test_x_control['cp_type'] == 'trt_cp'
treatment_flag = np.array([treatment_flag])
treatment_flag = np.repeat(treatment_flag.transpose(), 206, axis = 1)
out_holdout = out_holdout*treatment_flag
out_holdout_layers_1 = pd.DataFrame(data = out_holdout)

#Add columns name
out_holdout_layers_1.columns = train_y.columns
#Add column 'sig_id'
out_holdout_layers_1 = test_x_control[['sig_id']].join(out_holdout_layers_1)
#Submit
out_holdout_layers_1.to_csv('submission_1.csv', index = False)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train_layers_1))

-------------
Epoch 1:



100%|██████████| 549/549 [00:01<00:00, 468.70it/s]


Train loss: 0.039060 | Train accuracy: 0.993632


100%|██████████| 148/148 [00:00<00:00, 1612.62it/s]


Val loss:   0.020973 | Val accuracy:   0.996824
-------------
Epoch 2:



...

Train loss: 0.013788 | Train accuracy: 0.997251


100%|██████████| 148/148 [00:00<00:00, 1894.84it/s]


Val loss:   0.015650 | Val accuracy:   0.997282


100%|██████████| 1/1 [00:00<00:00, 23.30it/s]



Log loss train: 0.01374861440967069
Log loss dev: 0.01543760258601812


In [14]:
# Final model 2 hidden layers

X_train = train_x.to_numpy(copy = True)
Y_train = train_y.to_numpy(copy = True)

X_holdout = test_x.to_numpy(copy = True)   
X_holdout = torch.tensor(X_holdout, dtype=torch.float32)

n_epochs = 330               #Hyperparameter
batch_size = 32             #Hyperparameter
train_batches = batchify_data(X_train, Y_train, batch_size)
X_train_full = torch.tensor(X_train, dtype=torch.float32)



#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.5*X_train.shape[1])),         
                    nn.GELU(),
                    nn.Linear(int(0.5*X_train.shape[1]), int(0.4*X_train.shape[1])),     
                    nn.ReLU6(),
                    nn.Linear(int(0.4*X_train.shape[1]), 206),
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.11 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_2')

# Get output.
# Evaluate the model on test data.
out_train = model(X_train_full) 
out_holdout = model(X_holdout)  

out_train = out_train.detach().numpy()
out_train_layers_2 = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_holdout = out_holdout.detach().numpy()
# Make a matrix with zeros when control.
out_holdout = out_holdout*treatment_flag
out_holdout_layers_2 = pd.DataFrame(data = out_holdout)

#Add columns name
out_holdout_layers_2.columns = train_y.columns
#Add column 'sig_id'
out_holdout_layers_2 = test_x_control[['sig_id']].join(out_holdout_layers_2)
#Submit
out_holdout_layers_2.to_csv('submission_2.csv', index = False)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train_layers_2))

-------------
Epoch 1:



100%|██████████| 549/549 [00:01<00:00, 379.02it/s]


Train loss: 0.044678 | Train accuracy: 0.990049


100%|██████████| 148/148 [00:00<00:00, 1234.50it/s]


Val loss:   0.020267 | Val accuracy:   0.996745
-------------
Epoch 2:



...

Train loss: 0.014491 | Train accuracy: 0.997234


100%|██████████| 148/148 [00:00<00:00, 1184.26it/s]


Val loss:   0.015744 | Val accuracy:   0.997276


100%|██████████| 1/1 [00:00<00:00, 16.16it/s]



Log loss train: 0.014453019473714246
Log loss dev: 0.015533682865723701


In [15]:
# Final model 3 hidden layers

X_train = train_x.to_numpy(copy = True)
Y_train = train_y.to_numpy(copy = True)

X_holdout = test_x.to_numpy(copy = True)   
X_holdout = torch.tensor(X_holdout, dtype=torch.float32)

n_epochs = 310               #Hyperparameter
batch_size = 32             #Hyperparameter
train_batches = batchify_data(X_train, Y_train, batch_size)
X_train_full = torch.tensor(X_train, dtype=torch.float32)


#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.4*X_train.shape[1])), 
                    nn.SELU(),  
                    nn.Linear(int(0.4*X_train.shape[1]), int(1.3*X_train.shape[1])), 
                    nn.GELU(),
                    nn.Linear(int(1.3*X_train.shape[1]), int(0.5*X_train.shape[1])),  
                    nn.ReLU6(),
                    nn.Linear(int(0.5*X_train.shape[1]), 206),
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.09 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_3')

# Get output.
# Evaluate the model on test data.
out_train = model(X_train_full) 
out_holdout = model(X_holdout)  

out_train = out_train.detach().numpy()
out_train_layers_3 = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_holdout = out_holdout.detach().numpy()
# Make a matrix with zeros when control.
out_holdout = out_holdout*treatment_flag
out_holdout_layers_3 = pd.DataFrame(data = out_holdout)

#Add columns name
out_holdout_layers_3.columns = train_y.columns
#Add column 'sig_id'
out_holdout_layers_3 = test_x_control[['sig_id']].join(out_holdout_layers_3)
#Submit
out_holdout_layers_3.to_csv('submission_3.csv', index = False)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train_layers_3))

-------------
Epoch 1:



100%|██████████| 549/549 [00:02<00:00, 188.93it/s]


Train loss: 0.053258 | Train accuracy: 0.986397


100%|██████████| 148/148 [00:00<00:00, 694.55it/s]


Val loss:   0.020484 | Val accuracy:   0.996625
-------------
Epoch 2:



...

Train loss: 0.015609 | Train accuracy: 0.997148


100%|██████████| 148/148 [00:00<00:00, 753.76it/s]


Val loss:   0.015884 | Val accuracy:   0.997249


100%|██████████| 1/1 [00:00<00:00, 12.80it/s]



Log loss train: 0.015583818125281541
Log loss dev: 0.01567518146286339
